In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

conf = SparkConf().setMaster('local').setAppName('sparkApp')
spark = SparkContext(conf=conf)

sqlCtx = SQLContext(spark)
sqlCtx

### Titanic 실습

In [2]:
titanic=sqlCtx.read.csv('./data/spark_titanic_train.csv',
                       header=True,
                       inferSchema=True)
type(titanic)

pyspark.sql.dataframe.DataFrame

In [3]:
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
titanic.select(['Survived','Pclass']).show()

+--------+------+
|Survived|Pclass|
+--------+------+
|       0|     3|
|       1|     1|
|       1|     3|
|       1|     1|
|       0|     3|
|       0|     3|
|       0|     1|
|       0|     3|
|       1|     3|
|       1|     2|
|       1|     3|
|       1|     1|
|       0|     3|
|       0|     3|
|       0|     3|
|       1|     2|
|       0|     3|
|       1|     2|
|       0|     3|
|       1|     3|
+--------+------+
only showing top 20 rows



- EDA

In [5]:
titanic.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [6]:
titanic.groupBy('Sex','Survived').count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



In [7]:
from pyspark.sql.functions import mean,col,split,regexp_extract, when, lit


In [8]:
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
# This function use to print feature with null values and null count 
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)


In [9]:
null_list = null_value_count(titanic)

In [10]:
# 긱 feature에 대한 결측치의 수를 확인할 수 있다
null_list 

[('Age', 177), ('Cabin', 687), ('Embarked', 2)]

In [11]:
sqlCtx.createDataFrame(null_list,['column','cnt']).show()

+--------+---+
|  column|cnt|
+--------+---+
|     Age|177|
|   Cabin|687|
|Embarked|  2|
+--------+---+



In [12]:
# 나이에 대한 평균을 구한다면?
titanic.select(mean('Age')).show()

+-----------------+
|         avg(Age)|
+-----------------+
|29.69911764705882|
+-----------------+



In [13]:
titanic.select('Name').show()

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



In [16]:
# type(col('Name')) => column
# type('Name') => str
#regexp_extract(col('Name'))


# spark 컬럼 추가
titanic = titanic.withColumn('initial', regexp_extract(col('Name'), '([A-Za-z]+)\.', 1))

In [17]:
titanic.select('initial').show() # 추가 된 것을 확인 할 수 있다.

+-------+
|initial|
+-------+
|     Mr|
|    Mrs|
|   Miss|
|    Mrs|
|     Mr|
|     Mr|
|     Mr|
| Master|
|    Mrs|
|    Mrs|
|   Miss|
|   Miss|
|     Mr|
|     Mr|
|   Miss|
|    Mrs|
| Master|
|     Mr|
|    Mrs|
|    Mrs|
+-------+
only showing top 20 rows



In [18]:
titanic.select('initial').distinct().show()

+--------+
| initial|
+--------+
|     Don|
|    Miss|
|Countess|
|     Col|
|     Rev|
|    Lady|
|  Master|
|     Mme|
|    Capt|
|      Mr|
|      Dr|
|     Mrs|
|     Sir|
|Jonkheer|
|    Mlle|
|   Major|
|      Ms|
+--------+



In [19]:
titanic=titanic.replace(['Don'],['Other']) # 글자 변환


In [20]:
titanic.select('initial').distinct().show #확인

<bound method DataFrame.show of DataFrame[initial: string]>

In [22]:
titanic.groupby('initial').avg('age').show()

+--------+------------------+
| initial|          avg(age)|
+--------+------------------+
|    Miss|21.773972602739725|
|Countess|              33.0|
|     Col|              58.0|
|     Rev|43.166666666666664|
|    Lady|              48.0|
|   Other|              40.0|
|  Master| 4.574166666666667|
|     Mme|              24.0|
|    Capt|              70.0|
|      Mr|32.368090452261306|
|      Dr|              42.0|
|     Mrs|35.898148148148145|
|     Sir|              49.0|
|Jonkheer|              38.0|
|    Mlle|              24.0|
|   Major|              48.5|
|      Ms|              28.0|
+--------+------------------+



In [23]:
titanic = titanic.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs',  'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])


In [24]:
titanic.filter(titanic['Age']==48).select('initial').show()

+-------+
|initial|
+-------+
|     Mr|
|     Mr|
|    Mrs|
|     Mr|
|     Mr|
|    Mrs|
|    Mrs|
|     Mr|
|    Mrs|
+-------+



In [25]:
# null 처리 방법
titanic.groupby('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+



In [26]:
titanic = titanic.na.fill({'Embarked':'S'})

In [27]:
titanic.groupby('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|       C|  168|
|       S|  646|
+--------+-----+



In [28]:
titanic = titanic.drop('Cabin')

In [29]:
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- initial: string (nullable = true)



In [31]:
# 파생 컬럼 만든느 방법
titanic = titanic.withColumn('Family_Size',col('SibSp')+col('Parch'))

In [32]:
titanic.select('Family_Size').show()

+-----------+
|Family_Size|
+-----------+
|          1|
|          1|
|          0|
|          1|
|          0|
|          0|
|          0|
|          4|
|          2|
|          1|
|          2|
|          0|
|          0|
|          6|
|          0|
|          0|
|          5|
|          0|
|          1|
|          0|
+-----------+
only showing top 20 rows



In [34]:
titanic.groupby('Family_Size').count().show()

+-----------+-----+
|Family_Size|count|
+-----------+-----+
|          1|  161|
|          6|   12|
|          3|   29|
|          5|   22|
|          4|   15|
|          7|    6|
|         10|    7|
|          2|  102|
|          0|  537|
+-----------+-----+



In [37]:
# 0으로 셋팅
titanic = titanic.withColumn('Alona',lit(0))

In [39]:
titanic.select('Alona').show()

+-----+
|Alona|
+-----+
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
+-----+
only showing top 20 rows



In [41]:
titanic = titanic.withColumn('Alone',when(titanic['Family_Size']==0,1).otherwise(titanic['Alona']))

In [42]:
titanic.select('Alona').show()

+-----+
|Alona|
+-----+
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
|    0|
+-----+
only showing top 20 rows



In [43]:
titanic.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Embarked',
 'initial',
 'Family_Size',
 'Alona',
 'Alone']

In [44]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline


In [45]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(titanic) for column in ["Sex","Embarked","initial"]]
pipeline = Pipeline(stages=indexers)
titanic = pipeline.fit(titanic).transform(titanic)


In [46]:
titanic = titanic.na.fill({'Age':20})

In [47]:
titanic = titanic.drop("PassengerId","Name","Ticket","Cabin","Embarked","Sex","initial")


In [51]:
from pyspark.ml.feature import VectorAssembler

In [52]:
feature = VectorAssembler(inputCols = titanic.columns[1:],outputCol='features')
feature_vector=feature.transform(titanic)

In [53]:
feature_vector.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Family_Size: integer (nullable = true)
 |-- Alona: integer (nullable = false)
 |-- Alone: integer (nullable = false)
 |-- Sex_index: double (nullable = false)
 |-- Embarked_index: double (nullable = false)
 |-- initial_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [54]:
trainData,testData = feature_vector.randomSplit([.8,.2],seed=100)

In [55]:
trainData.show()

+--------+------+----+-----+-----+--------+-----------+-----+-----+---------+--------------+-------------+--------------------+
|Survived|Pclass| Age|SibSp|Parch|    Fare|Family_Size|Alona|Alone|Sex_index|Embarked_index|initial_index|            features|
+--------+------+----+-----+-----+--------+-----------+-----+-----+---------+--------------+-------------+--------------------+
|       0|     1| 2.0|    1|    2|  151.55|          3|    0|    0|      1.0|           0.0|          1.0|[1.0,2.0,1.0,2.0,...|
|       0|     1|18.0|    1|    0|   108.9|          1|    0|    0|      0.0|           1.0|          0.0|(11,[0,1,2,4,5,9]...|
|       0|     1|19.0|    1|    0|    53.1|          1|    0|    0|      0.0|           0.0|          0.0|(11,[0,1,2,4,5],[...|
|       0|     1|20.0|    0|    0|     0.0|          0|    0|    1|      0.0|           0.0|          0.0|(11,[0,1,7],[1.0,...|
|       0|     1|20.0|    0|    0|     0.0|          0|    0|    1|      0.0|           0.0|          0.

- 모델링
- Spark ML(DTC, LR, RFC,COTC,NB, SVM)

In [65]:
# LOGISTIC REGRESSION
# 데이터 범주가 0,1 사이의 값으로 예측하는 분류 알고리즘
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol='Survived',featuresCol='features')
lr_model = lr.fit(trainData) # 학습된 모델 만들기 (학습이 이루어짐)
lr_pred = lr_model.transform(testData)

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\ml\wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LogisticRegression' object has no attribute '_java_obj'
Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\ml\wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LogisticRegression' object has no attribute '_java_obj'
Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\ml\wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'LogisticRegression' object has no attribute '_java_obj'


In [69]:
lr_pred.show()

+--------+------+----+-----+-----+-------+-----------+-----+-----+---------+--------------+-------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alona|Alone|Sex_index|Embarked_index|initial_index|            features|       rawPrediction|         probability|prediction|
+--------+------+----+-----+-----+-------+-----------+-----+-----+---------+--------------+-------------+--------------------+--------------------+--------------------+----------+
|       0|     1|19.0|    3|    2|  263.0|          5|    0|    0|      0.0|           0.0|          0.0|(11,[0,1,2,3,4,5]...|[1.12323145345704...|[0.75458762580674...|       0.0|
|       0|     1|20.0|    0|    0|27.7208|          0|    0|    1|      0.0|           1.0|          0.0|(11,[0,1,4,7,9],[...|[0.16165256257039...|[0.54032536516929...|       0.0|
|       0|     1|20.0|    0|    0|   39.6|          0|    0|    1|      0.0|           1.0|         

In [72]:
# type(lr_pred) -> 데이터 프레임
# lr_pred.printSchema()
lr_pred.select('Prediction','Survived','features').show()

+----------+--------+--------------------+
|Prediction|Survived|            features|
+----------+--------+--------------------+
|       0.0|       0|(11,[0,1,2,3,4,5]...|
|       0.0|       0|(11,[0,1,4,7,9],[...|
|       0.0|       0|(11,[0,1,4,7,9],[...|
|       1.0|       0|(11,[0,1,3,4,5],[...|
|       1.0|       0|(11,[0,1,3,4,5,9]...|
|       0.0|       0|(11,[0,1,2,4,5],[...|
|       0.0|       0|(11,[0,1,4,7,9],[...|
|       0.0|       0|(11,[0,1,2,4,5],[...|
|       0.0|       0|(11,[0,1,2,4,5],[...|
|       1.0|       0|(11,[0,1,7,10],[1...|
|       0.0|       0|(11,[0,1,7],[1.0,...|
|       0.0|       0|(11,[0,1,7],[1.0,...|
|       0.0|       0|(11,[0,1,4,7],[1....|
|       0.0|       0|(11,[0,1,2,4,5,9]...|
|       0.0|       0|(11,[0,1,3,4,5,9]...|
|       0.0|       0|(11,[0,1,4,7,9],[...|
|       0.0|       0|(11,[0,1,4,7],[1....|
|       0.0|       0|(11,[0,1,4,7,9],[...|
|       0.0|       0|(11,[0,1,4,7],[2....|
|       0.0|       0|(11,[0,1,2,4,5],[...|
+----------

In [74]:
# 평가
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluater = MulticlassClassificationEvaluator(labelCol='Survived',
                                    predictionCol='prediction',
                                    metricName='accuracy')

In [75]:
acc = evaluater.evaluate(lr_pred)
print('acc : ',acc)
print('err : ',1.0-acc)

acc :  0.8
err :  0.19999999999999996
